#### 提升方向：
- 止盈和止损点位
- 将RSI window dynamic


#### 止盈设置
- ATR Rachket
- 将止盈从原先的固定0.05变为浮动
     - 持仓周期
     - ATR波动挂钩 
- 观察不同阈值下触及止盈和止损的位置

#### 噪音观察
- 解释DrawBack的原因


In [1]:
import sys
sys.path.append('D:/vnpy_strategies/DoubleRsi')
import numpy as np
import pandas as pd
from vnpy.app.cta_strategy.backtesting import BacktestingEngine, OptimizationSetting
from datetime import datetime
from double_rsi_atr import DoubleRsiAtr
from rsi_dollor import RsiDollar
from double_rsi_signal import DoubleSignalGenerator
from trade_by_trade import (calculate_trades_result, generate_trade_df)
import matplotlib.pyplot as plt

In [2]:
start_time = datetime(2010, 10, 1)
end_time = datetime(2019, 4, 30)

In [ ]:
strategy_parameters={'s_window':5,'l_window':15,'atr_window':35,'atr_multiplier':0.25,'exit_return_soft_long':-0.1,
                     'obv_window':35,'minus_dm_window':35,'mfi_window':35,'ad_window':35,'adosc_window':35,'plus_dm_window':35,
                     'dx_window':35,'adx_window':35,
                     'exit_loss_soft_long':-0.2,'rsi_window':11,'exit_return_soft_short':0.05,'exit_loss_soft_short':0.3,
                     'long_threshold_l_window': 50, 'long_threshold_s_window':80,'exit_return':0.09,'exit_loss':0.01,'fixed_size':1}


In [ ]:
engine = BacktestingEngine()
engine.set_parameters(
    vt_symbol="IF888.CFFEX",
    interval="1m",
    start= start_time,
    end= end_time,
    rate=5/10000,
    slippage=0.6,
    size=300,
    pricetick=0.2,
    capital=1_000_000,
)
engine.add_strategy(DoubleRsiAtr, strategy_parameters)

In [ ]:
engine.load_data()
engine.run_backtesting()
df = engine.calculate_result()
summary_dict = engine.calculate_statistics()

In [ ]:
engine.show_chart()

In [ ]:
engine = BacktestingEngine()
engine.set_parameters(
    vt_symbol="IF888.CFFEX",
    interval="1m",
    start = start_time,
    end = end_time,
    rate=5/10000,
    slippage=0.6,
    size=300,
    pricetick=0.2,
    capital=1_000_000,
)
engine.add_strategy(DoubleSignalGenerator, strategy_parameters)
engine.load_data()
engine.run_backtesting()
df = engine.calculate_result()
summary_dict = engine.calculate_statistics()

In [ ]:
rsi_value_l_window = pd.Series(engine.strategy.rsi_value_recorder_l_window,index=engine.strategy.rsi_time_recorder_l_window)
rsi_value_s_window =pd.Series(engine.strategy.rsi_value_recorder_s_window,index=engine.strategy.rsi_time_recorder_s_window)
atr_value = pd.Series(engine.strategy.atr_value_recorder,index=engine.strategy.atr_time_recorder)
obv_value = pd.Series(engine.strategy.obv_value_recorder,index=engine.strategy.obv_time_recorder)
minus_dm_value = pd.Series(engine.strategy.minus_dm_value_recorder,index=engine.strategy.minus_dm_time_recorder)
mfi_value = pd.Series(engine.strategy.mfi_value_recorder,index=engine.strategy.mfi_time_recorder)
ad_value = pd.Series(engine.strategy.ad_value_recorder,index=engine.strategy.ad_time_recorder)
adosc_value = pd.Series(engine.strategy.adosc_value_recorder,index=engine.strategy.adosc_time_recorder)
plus_dm_value =pd.Series(engine.strategy.plus_dm_value_recorder,index=engine.strategy.plus_dm_time_recorder)
dx_value = pd.Series(engine.strategy.dx_value_recorder,index=engine.strategy.dx_time_recorder)
adx_value = pd.Series(engine.strategy.adx_value_recorder,index=engine.strategy.adx_time_recorder)

In [ ]:
indicator_df = pd.concat([rsi_value_s_window,rsi_value_l_window.reindex(rsi_value_s_window.index,method='ffill'),atr_value,obv_value,
                          minus_dm_value,mfi_value,ad_value,adosc_value,plus_dm_value,dx_value,adx_value],axis=1)
indicator_df.columns = ['rsi_short_window','rsi_long_window','atr','obv','minus_dm','mfi','ad','adosc','plus_dm','dx','adx']
indicator_df.dropna(inplace=True)

### 最大回撤区间
- 2013-2015
- 2016-2017

### 最小回撤区间
- 2011 -2013

In [ ]:
fig = plt.figure(figsize=(25,10))

plt.plot(indicator_df['atr']['2011-01-01':'2013-12-31'],'g-',label='slightly_draw_back_period')
plt.plot(indicator_df['atr']['2014-01-01':'2015-01-01'],'r-',label='heavily_draw_back_period')
plt.plot(indicator_df['atr']['2015-01-01':'2015-12-31'],'g-')
plt.plot(indicator_df['atr']['2016-01-01':'2016-12-31'],'r-')
plt.plot(indicator_df['atr']['2017-01-01':'2017-12-31'],'g-')
plt.plot(indicator_df['atr']['2018-01-01':'2018-12-31'],'r-')
plt.plot(indicator_df['atr']['2019-01-01':'2019-12-31'],'g-')
plt.legend()
# # ax1.plot(long_accuracy_with_fee,'r-',label='long_side_accuracy')
# ax2.plot(two_pnl_df['close'],'b--',label='close')
# plt.legend()


### Indicator 和 策略 DrawBack
- 将DrawBack分成两个阶段：*Slightly DrawBack，Heavily DrawBack*
    - *RSI* 指标没办法区别这两个*DrawBack* period
    - *ATR* 指标似乎对这两个*DrawBack* period 有一定程度的区分
    

In [ ]:
fig = plt.figure(figsize=(25,10))

plt.plot(indicator_df['rsi_short_window']['2011-01-01':'2013-12-31'],'g-',label='slightly_draw_back_period')
plt.plot(indicator_df['rsi_short_window']['2014-01-01':'2015-01-01'],'r-',label='heavily_draw_back_period')
plt.plot(indicator_df['rsi_short_window']['2015-01-01':'2015-12-31'],'g-')
plt.plot(indicator_df['rsi_short_window']['2016-01-01':'2016-12-31'],'r-')
plt.plot(indicator_df['rsi_short_window']['2017-01-01':'2017-12-31'],'g-')
plt.plot(indicator_df['rsi_short_window']['2018-01-01':'2018-12-31'],'r-')
plt.plot(indicator_df['rsi_short_window']['2019-01-01':'2019-12-31'],'g-')
plt.legend()

In [ ]:
fig = plt.figure(figsize=(25,10))

plt.plot(indicator_df['rsi_long_window']['2011-01-01':'2013-12-31'],'g-',label='slightly_draw_back_period')
plt.plot(indicator_df['rsi_long_window']['2014-01-01':'2015-01-01'],'r-',label='heavily_draw_back_period')
plt.plot(indicator_df['rsi_long_window']['2015-01-01':'2015-12-31'],'g-')
plt.plot(indicator_df['rsi_long_window']['2016-01-01':'2016-12-31'],'r-')
plt.plot(indicator_df['rsi_long_window']['2017-01-01':'2017-12-31'],'g-')
plt.plot(indicator_df['rsi_long_window']['2018-01-01':'2018-12-31'],'r-')
plt.plot(indicator_df['rsi_long_window']['2019-01-01':'2019-12-31'],'g-')
plt.legend()

In [ ]:
fig = plt.figure(figsize=(25,10))

plt.plot(indicator_df['obv']['2011-01-01':'2013-12-31'],'g-',label='slightly_draw_back_period')
plt.plot(indicator_df['obv']['2014-01-01':'2015-01-01'],'r-',label='heavily_draw_back_period')
plt.plot(indicator_df['obv']['2015-01-01':'2015-12-31'],'g-')
plt.plot(indicator_df['obv']['2016-01-01':'2016-12-31'],'r-')
plt.plot(indicator_df['obv']['2017-01-01':'2017-12-31'],'g-')
plt.plot(indicator_df['obv']['2018-01-01':'2018-12-31'],'r-')
plt.plot(indicator_df['obv']['2019-01-01':'2019-12-31'],'g-')
plt.legend()

In [ ]:
fig = plt.figure(figsize=(25,10))

plt.plot(indicator_df['minus_dm']['2011-01-01':'2013-12-31'],'g-',label='slightly_draw_back_period')
plt.plot(indicator_df['minus_dm']['2014-01-01':'2015-01-01'],'r-',label='heavily_draw_back_period')
plt.plot(indicator_df['minus_dm']['2015-01-01':'2015-12-31'],'g-')
plt.plot(indicator_df['minus_dm']['2016-01-01':'2016-12-31'],'r-')
plt.plot(indicator_df['minus_dm']['2017-01-01':'2017-12-31'],'g-')
plt.plot(indicator_df['minus_dm']['2018-01-01':'2018-12-31'],'r-')
plt.plot(indicator_df['minus_dm']['2019-01-01':'2019-12-31'],'g-')
plt.legend()

In [ ]:

fig = plt.figure(figsize=(25,10))

plt.plot(indicator_df['mfi']['2011-01-01':'2013-12-31'],'g-',label='slightly_draw_back_period')
plt.plot(indicator_df['mfi']['2014-01-01':'2015-01-01'],'r-',label='heavily_draw_back_period')
plt.plot(indicator_df['mfi']['2015-01-01':'2015-12-31'],'g-')
plt.plot(indicator_df['mfi']['2016-01-01':'2016-12-31'],'r-')
plt.plot(indicator_df['mfi']['2017-01-01':'2017-12-31'],'g-')
plt.plot(indicator_df['mfi']['2018-01-01':'2018-12-31'],'r-')
plt.plot(indicator_df['mfi']['2019-01-01':'2019-12-31'],'g-')
plt.legend()

In [ ]:

fig = plt.figure(figsize=(25,10))

plt.plot(indicator_df['ad']['2011-01-01':'2013-12-31'],'g-',label='slightly_draw_back_period')
plt.plot(indicator_df['ad']['2014-01-01':'2015-01-01'],'r-',label='heavily_draw_back_period')
plt.plot(indicator_df['ad']['2015-01-01':'2015-12-31'],'g-')
plt.plot(indicator_df['ad']['2016-01-01':'2016-12-31'],'r-')
plt.plot(indicator_df['ad']['2017-01-01':'2017-12-31'],'g-')
plt.plot(indicator_df['ad']['2018-01-01':'2018-12-31'],'r-')
plt.plot(indicator_df['ad']['2019-01-01':'2019-12-31'],'g-')
plt.legend()

In [ ]:
fig = plt.figure(figsize=(25,10))

plt.plot(indicator_df['adosc']['2011-01-01':'2013-12-31'],'g-',label='slightly_draw_back_period')
plt.plot(indicator_df['adosc']['2014-01-01':'2015-01-01'],'r-',label='heavily_draw_back_period')
plt.plot(indicator_df['adosc']['2015-01-01':'2015-12-31'],'g-')
plt.plot(indicator_df['adosc']['2016-01-01':'2016-12-31'],'r-')
plt.plot(indicator_df['adosc']['2017-01-01':'2017-12-31'],'g-')
plt.plot(indicator_df['adosc']['2018-01-01':'2018-12-31'],'r-')
plt.plot(indicator_df['adosc']['2019-01-01':'2019-12-31'],'g-')
plt.legend()

In [ ]:
fig = plt.figure(figsize=(25,10))

plt.plot(indicator_df['plus_dm']['2011-01-01':'2013-12-31'],'g-',label='slightly_draw_back_period')
plt.plot(indicator_df['plus_dm']['2014-01-01':'2015-01-01'],'r-',label='heavily_draw_back_period')
plt.plot(indicator_df['plus_dm']['2015-01-01':'2015-12-31'],'g-')
plt.plot(indicator_df['plus_dm']['2016-01-01':'2016-12-31'],'r-')
plt.plot(indicator_df['plus_dm']['2017-01-01':'2017-12-31'],'g-')
plt.plot(indicator_df['plus_dm']['2018-01-01':'2018-12-31'],'r-')
plt.plot(indicator_df['plus_dm']['2019-01-01':'2019-12-31'],'g-')
plt.legend()

In [ ]:
fig = plt.figure(figsize=(25,10))

plt.plot(indicator_df['dx']['2011-01-01':'2013-12-31'],'g-',label='slightly_draw_back_period')
plt.plot(indicator_df['dx']['2014-01-01':'2015-01-01'],'r-',label='heavily_draw_back_period')
plt.plot(indicator_df['dx']['2015-01-01':'2015-12-31'],'g-')
plt.plot(indicator_df['dx']['2016-01-01':'2016-12-31'],'r-')
plt.plot(indicator_df['dx']['2017-01-01':'2017-12-31'],'g-')
plt.plot(indicator_df['dx']['2018-01-01':'2018-12-31'],'r-')
plt.plot(indicator_df['dx']['2019-01-01':'2019-12-31'],'g-')
plt.legend()



In [ ]:
fig = plt.figure(figsize=(25,10))

plt.plot(indicator_df['adx']['2011-01-01':'2013-12-31'],'g-',label='slightly_draw_back_period')
plt.plot(indicator_df['adx']['2014-01-01':'2015-01-01'],'r-',label='heavily_draw_back_period')
plt.plot(indicator_df['adx']['2015-01-01':'2015-12-31'],'g-')
plt.plot(indicator_df['adx']['2016-01-01':'2016-12-31'],'r-')
plt.plot(indicator_df['adx']['2017-01-01':'2017-12-31'],'g-')
plt.plot(indicator_df['adx']['2018-01-01':'2018-12-31'],'r-')
plt.plot(indicator_df['adx']['2019-01-01':'2019-12-31'],'g-')
plt.legend()

In [ ]:
np.sum([1,2,3])

In [3]:
start_time = datetime(2010, 10, 1)
end_time = datetime(2013, 4, 30)

In [4]:
engine = BacktestingEngine()
engine.set_parameters(
    vt_symbol="IF888.CFFEX",
    interval="1m",
    start = start_time,
    end = end_time,
    rate=5/10000,
    slippage=0.6,
    size=300,
    pricetick=0.2,
    capital=1_000_000,
)
engine.add_strategy(RsiDollar,{})
engine.load_data()
engine.run_backtesting()
df = engine.calculate_result()
summary_dict = engine.calculate_statistics()

2020-04-17 22:47:04.469939	开始加载历史数据
2020-04-17 22:47:04.676245	加载进度： [3%]
2020-04-17 22:47:04.926537	加载进度： [6%]
2020-04-17 22:47:05.219525	加载进度： [10%]
2020-04-17 22:47:05.466134	加载进度：# [13%]
2020-04-17 22:47:05.656277	加载进度：# [16%]
2020-04-17 22:47:05.966408	加载进度：# [19%]
2020-04-17 22:47:06.202589	加载进度：## [22%]
2020-04-17 22:47:06.483523	加载进度：## [25%]
2020-04-17 22:47:06.736802	加载进度：## [29%]
2020-04-17 22:47:07.079055	加载进度：### [32%]
2020-04-17 22:47:07.370204	加载进度：### [35%]
2020-04-17 22:47:07.603112	加载进度：### [38%]
2020-04-17 22:47:07.891743	加载进度：#### [41%]
2020-04-17 22:47:08.146497	加载进度：#### [45%]
2020-04-17 22:47:08.403601	加载进度：#### [48%]
2020-04-17 22:47:08.635602	加载进度：##### [51%]
2020-04-17 22:47:08.931835	加载进度：##### [54%]
2020-04-17 22:47:09.214902	加载进度：##### [57%]
2020-04-17 22:47:09.429035	加载进度：###### [61%]
2020-04-17 22:47:09.762883	加载进度：###### [64%]
2020-04-17 22:47:10.023360	加载进度：###### [67%]
2020-04-17 22:47:10.271635	加载进度：####### [70%]
2020-04-17 22:47:10.521534	加载进度：######

In [ ]:
engine.strategy.

In [ ]:
import talib


In [ ]:
talib.BBANDS

In [ ]:
rsi_value_l_window = pd.Series(engine.strategy.rsi_value_recorder_l_window,index=engine.strategy.rsi_time_recorder_l_window)
rsi_value_s_window =pd.Series(engine.strategy.rsi_value_recorder_s_window,index=engine.strategy.rsi_time_recorder_s_window)
atr_value = pd.Series(engine.strategy.atr_value_recorder,index=engine.strategy.atr_time_recorder)
obv_value = pd.Series(engine.strategy.obv_value_recorder,index=engine.strategy.obv_time_recorder)
minus_dm_value = pd.Series(engine.strategy.minus_dm_value_recorder,index=engine.strategy.minus_dm_time_recorder)
mfi_value = pd.Series(engine.strategy.mfi_value_recorder,index=engine.strategy.mfi_time_recorder)
ad_value = pd.Series(engine.strategy.ad_value_recorder,index=engine.strategy.ad_time_recorder)
adosc_value = pd.Series(engine.strategy.adosc_value_recorder,index=engine.strategy.adosc_time_recorder)
plus_dm_value =pd.Series(engine.strategy.plus_dm_value_recorder,index=engine.strategy.plus_dm_time_recorder)
dx_value = pd.Series(engine.strategy.dx_value_recorder,index=engine.strategy.dx_time_recorder)
adx_value = pd.Series(engine.strategy.adx_value_recorder,index=engine.strategy.adx_time_recorder)